<a href="https://colab.research.google.com/github/RodrigodeFeo/8DTS/blob/main/Aula_5_DTS_PLN_Exerc%C3%ADcio_corrigido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Exercício - Aula 5**

## ***Exercício***

Você deverá treinar o modelo de classificação do dataset de produtos [1] utilizando as duas abordagens apresentadas aqui (embeddings word2vec (CBOW ou skip-gram) e sentence transformer) e comparar os resultados.

*   Remover registros com valores nulos;
*   Contatenar as colunas de nome e descrição;
*   Aplicar normalização da demo;
*   Amostra de 30% para teste e random_state = 42.


[1] https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv

### **Classificador com word2vec**

In [ ]:
# Download do arquivo no repositório do professor
!wget 'https://dados-ml-pln.s3-sa-east-1.amazonaws.com/skip_s300.zip'

# Descompactação do arquivo
!unzip 'skip_s300.zip' # subistitua com nome do arquivo
!ls -la

--2023-12-02 01:10:09--  https://dados-ml-pln.s3-sa-east-1.amazonaws.com/skip_s300.zip
Resolving dados-ml-pln.s3-sa-east-1.amazonaws.com (dados-ml-pln.s3-sa-east-1.amazonaws.com)... 52.95.163.126, 16.12.1.62, 3.5.234.115, ...
Connecting to dados-ml-pln.s3-sa-east-1.amazonaws.com (dados-ml-pln.s3-sa-east-1.amazonaws.com)|52.95.163.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 958619745 (914M) [application/zip]
Saving to: ‘skip_s300.zip’

skip_s300.zip       100%[===================>] 914.21M  25.6MB/s    in 36s     

2023-12-02 01:10:46 (25.1 MB/s) - ‘skip_s300.zip’ saved [958619745/958619745]

Archive:  skip_s300.zip
  inflating: skip_s300.txt           
total 3529584
drwxr-xr-x 1 root root       4096 Dec  2 01:10 .
drwxr-xr-x 1 root root       4096 Dec  2 01:09 ..
drwxr-xr-x 4 root root       4096 Nov 30 14:24 .config
drwxr-xr-x 1 root root       4096 Nov 30 14:27 sample_data
-rw-r--r-- 1 root root 2655642222 Oct  4  2018 skip_s300.txt
-rw-r--r-- 1 root

In [ ]:
# Load do modelo pelo Gensim
from gensim.models import KeyedVectors

model_skip = KeyedVectors.load_word2vec_format('skip_s300.txt')

In [ ]:
model_skip

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Vamos usar nosso dataframe de produtos
df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')
df.dropna(inplace=True)
df["texto"] = df['nome'] + " " + df['descricao']

In [ ]:
# Normalização de texto
import string

import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

# lista de stopwords do NLTK
stopwords = nltk.corpus.stopwords.words('portuguese')

# função que remove pontuação
def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

# função que normaliza o texto e remove stopwords
def norm_tokenize(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = "".join([w for w in text if not w.isdigit()])
    text = word_tokenize(text)
    text = [x for x in text if x not in stopwords]
    text = [y for y in text if len(y) > 2]
    #return " ".join([t for t in text])
    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Tokenizando e normalizando o texto
df['tokens'] = df['texto'].apply(norm_tokenize)

In [ ]:
# divisão da amostra entre treino e teste
df_train, df_test = train_test_split(
      df,
      test_size = 0.3,
      random_state = 42
  )

In [ ]:
# Função para obter embeddings médios para cada texto
vectorizer = model_skip

def average_vector(words):
  vectors = [vectorizer[word] for word in words if word in vectorizer]
  if vectors:
    return sum(vectors) / len(vectors)
  else:
    vector_size = vectorizer.vector_size
    wv_res = np.zeros(vector_size)
    return wv_res

In [ ]:
# Converter textos para embeddings com Word2Vec
df_train["vetor"] = df_train["tokens"].apply(average_vector)

In [ ]:
# variáveis
x_train = df_train["vetor"]
y_train = df_train["categoria"]

# Treinar Classificador
model = LogisticRegression(random_state=42)
#model = RandomForestClassifier(random_state=42)
model.fit(list(x_train), y_train)

# Avaliação do modelo
df_test["vetor"] = df_test["tokens"].apply(average_vector)
x_test = df_test["vetor"]
y_test = df_test["categoria"]

y_pred = model.predict(list(x_test))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

   brinquedo       0.97      0.92      0.94       190
        game       0.94      0.98      0.96       194
       livro       0.98      0.98      0.98       261
   maquiagem       0.98      1.00      0.99       230

    accuracy                           0.97       875
   macro avg       0.97      0.97      0.97       875
weighted avg       0.97      0.97      0.97       875

0.9714285714285714


### **Classificador com Transformers**

In [ ]:
!pip install sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 844.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Vamos usar nosso dataframe de produtos
df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')
df.dropna(inplace=True)
df["texto"] = df['nome'] + " " + df['descricao']

In [ ]:
# Normalização de texto
import string
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

# lista de stopwords do NLTK
stopwords = nltk.corpus.stopwords.words('portuguese')

# função que remove pontuação
def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

# função que normaliza o texto e remove stopwords
def norm_tokenize(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = "".join([w for w in text if not w.isdigit()])
    text = word_tokenize(text)
    text = [x for x in text if x not in stopwords]
    text = [y for y in text if len(y) > 2]
    return " ".join([t for t in text])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Tokenizando e normalizando o texto
df['texto_trat'] = df['texto'].apply(norm_tokenize)

In [ ]:
from sentence_transformers import SentenceTransformer

st = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
vetor_embedding = st.encode(df['texto_trat'].to_list())

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

In [ ]:
df['vetor_st'] = list(vetor_embedding)
df.head()

,nome,descricao,categoria,texto,texto_trat,vetor_st
0,O Hobbit - 7ª Ed. 2013,Produto NovoBilbo Bolseiro é um hobbit que lev...,livro,O Hobbit - 7ª Ed. 2013 Produto NovoBilbo Bol...,hobbit produto novobilbo bolseiro hobbit leva ...,"[-0.0028997906, 0.02313526, 0.07833927, -0.045..."
1,Livro - It A Coisa - Stephen King,Produto NovoDurante as férias escolares de 195...,livro,Livro - It A Coisa - Stephen King Produto No...,livro coisa stephen king produto novodurante f...,"[-0.012316837, 0.0409282, 0.050890353, -0.0625..."
2,Box As Crônicas De Gelo E Fogo Pocket 5 Li...,Produto NovoTodo o reino de Westeros ao alcanc...,livro,Box As Crônicas De Gelo E Fogo Pocket 5 Li...,box crônicas gelo fogo pocket livros produto n...,"[0.006764433, 0.021254232, 0.019002922, -0.038..."
3,Box Harry Potter,Produto Novo e Físico A série Harry Potter ch...,livro,Box Harry Potter Produto Novo e Físico A sé...,box harry potter produto novo físico série har...,"[0.000646104, 0.061039712, 0.036964573, -0.050..."
4,Livro Origem - Dan Brown,Produto NovoDe Onde Viemos? Para Onde Vamos? R...,livro,Livro Origem - Dan Brown Produto NovoDe Onde...,livro origem dan brown produto novode onde vie...,"[-0.0067804162, 0.029927477, -0.014554762, -0...."


In [ ]:
# divisão da amostra entre treino e teste
df_train, df_test = train_test_split(
      df,
      test_size = 0.3,
      random_state = 42
  )

In [ ]:
len(vetor_embedding[0])

512

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score, confusion_matrix

# variáveis
x_train = df_train['vetor_st'].to_list()
y_train = df_train['categoria'].to_list()

# Treinar Classificador
model = LogisticRegression(random_state=42)
model.fit(x_train, y_train)

# Avaliação do modelo
x_test = df_test['vetor_st'].to_list()
y_test = df_test['categoria'].to_list()

y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

   brinquedo       0.96      0.95      0.95       190
        game       0.97      0.97      0.97       194
       livro       0.99      0.97      0.98       261
   maquiagem       0.97      1.00      0.98       230

    accuracy                           0.97       875
   macro avg       0.97      0.97      0.97       875
weighted avg       0.97      0.97      0.97       875

0.9737142857142858


RandomForestClassifier

In [ ]:
# variáveis
x_train = df_train['vetor_st'].to_list()
y_train = df_train['categoria'].to_list()

# Treinar Classificador
model = RandomForestClassifier(random_state=42)
model.fit(x_train, y_train)

# Avaliação do modelo
x_test = df_test['vetor_st'].to_list()
y_test = df_test['categoria'].to_list()

y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

   brinquedo       0.95      0.93      0.94       190
        game       0.97      0.97      0.97       194
       livro       0.98      0.96      0.97       261
   maquiagem       0.96      1.00      0.98       230

    accuracy                           0.96       875
   macro avg       0.96      0.96      0.96       875
weighted avg       0.96      0.96      0.96       875

0.9645714285714285


##Extra!

Mais exemplos de como trabalhar com feature engineering com modelos generativos.

### Classificação a partir do embedding do contexto do dataset

Podemos criar um embedding aplicando o werd2vec a partir do dataset que estamos usando e classificar nosso modelo pelo embedding genérico que criamos do dataset.

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Vamos usar nosso dataframe de produtos
df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')
df.dropna(inplace=True)
df["texto"] = df['nome'] + " " + df['descricao']

In [ ]:
# Normalização de texto
import string

import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

# lista de stopwords do NLTK
stopwords = nltk.corpus.stopwords.words('portuguese')

# função que remove pontuação
def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

# função que normaliza o texto e remove stopwords
def norm_tokenize(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = "".join([w for w in text if not w.isdigit()])
    text = word_tokenize(text)
    text = [x for x in text if x not in stopwords]
    text = [y for y in text if len(y) > 2]
    #return " ".join([t for t in text])
    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Tokenizando e normalizando o texto
df['tokens'] = df['texto'].apply(norm_tokenize)

In [ ]:
# divisão da amostra entre treino e teste
df_train, df_test = train_test_split(
      df,
      test_size = 0.3,
      random_state = 42
  )

In [ ]:
# Treinando nosso modelo com o nosso vocabulário
w2v_produto = Word2Vec(df_train['tokens'], vector_size=100, min_count=5, sg=0)

# Treina o modelo. Note que estamos setando o parametro sd = 0, ou seja, o algoritmo utilizado sera o CBOW
# Outro ponto de destaque é que quando setamos o parametro min_count para 1 estamos forcando o uso de todas as palavras.

In [ ]:
# Função para obter embeddings médios para cada texto
def average_vector(words):
    vectors = [w2v_produto.wv[word] for word in words if word in w2v_produto.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return []

In [ ]:
# Converter textos para embeddings com Word2Vec
df_train["vetor"] = df_train["tokens"].apply(average_vector)

In [ ]:
# variáveis
x_train = df_train["vetor"]
y_train = df_train["categoria"]

# Treinar Classificador
model = RandomForestClassifier(random_state=42)
model.fit(list(x_train), y_train)

# Resultados
df_test["vetor"] = df_test["tokens"].apply(average_vector)
x_test = df_test["vetor"]
y_test = df_test["categoria"]

y_pred = model.predict(list(x_test))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

   brinquedo       0.88      0.97      0.92       190
        game       0.97      0.95      0.96       194
       livro       0.99      0.96      0.97       261
   maquiagem       1.00      0.97      0.98       230

    accuracy                           0.96       875
   macro avg       0.96      0.96      0.96       875
weighted avg       0.96      0.96      0.96       875

0.96
